In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
import pickle

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
#Open Data File
name_cs_str = "CS1"
theta_dim = 2 #How to get this beforehand?
methods = ["A1", "B1", "A2", "B2", "C2"]
ep_methods = ["CONSTANT", "DECAY", "BOYLE", "JASRASARIA"]

#Loop over methods
for i in range(len(methods)):
    #Initialize overall min params
    sse_min_all = np.inf
    theta_min_all = np.zeros((1,theta_dim))
    run_all = 0
    iter_all = 0
    best_ep_meth = "None"
    #Loop over ep_methods
    for j in range(len(ep_methods)):
        #Pull out file
        fileObj = open("2023/08/28/Data_Files/" + name_cs_str + "_BO_method_" + methods[i] + "_ep_method_" + ep_methods[j] 
               + ".pickle", 'rb')
        results = pickle.load(fileObj)        

        tot_runs = results[run_num].configuration["Number of Workflow Restarts"]
        #Loop over runs
        for k in range(tot_runs):
            run_results = results[k].results_df
            #Find lowest sse and corresponding theta
            min_sse_index = np.argmin(run_results['Min Obj'])
            min_sse = run_results['Min Obj'].iloc[min_sse_index]
            min_sse_theta = run_results['Theta Min Obj'].iloc[min_sse_index]
            
            if min_sse < sse_min_all:
                sse_min_all = min_sse
                theta_min_all = min_sse_theta
                iter_all = min_sse_index
                run_all = k
                best_ep_meth = ep_methods[j]
        #Close file
        fileObj.close()
        #Print the lowest sse overall and corresponding theta
    print(methods[i] ,"\n", best_ep_meth ,"\n", sse_min_all, theta_min_all, run_all + 1, iter_all +1)
    

A1 
 CONSTANT 
 -12.34333908786287 [ 2.         -1.36504165] 4 1
B1 
 CONSTANT 
 0.023183805991544206 [ 0.9106293  -1.46383318] 10 31
A2 
 DECAY 
 2.254331280520984e-07 [ 1.00070633 -1.00005245] 6 6
B2 
 CONSTANT 
 1.4160430385827246e-09 [ 1.00006104 -1.00000309] 5 32
C2 
 BOYLE 
 4.9950314280363136e-09 [ 0.99987805 -0.99998925] 10 33
